In [12]:
from Cuttings import *

import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import time
import copy
import PIL.Image as Image
import random

import torch.optim as optim
import torchvision.transforms as tf
from torch.utils import data 
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import pickle 
from tqdm import tqdm

In [13]:
# torchvision for pre-trained models
from torchvision import models

#### Classes

In [14]:
class MinMaxNormalization(object):
    """
    Normalized (Min-Max) the image.
    """
    def __init__(self, vmin=0, vmax=1):
        """
        Constructor of the grayscale transform.
        ----------
        INPUT
            |---- vmin (float / int) the desired minimum value.
            |---- vmax (float / int) the desired maximum value.
        OUTPUT
            |---- None
        """
        self.vmin = vmin
        self.vmax = vmax

    def __call__(self, image, mask=None):
        """
        Apply a Min-Max Normalization to the image.
        ----------
        INPUT
            |---- image (PIL.Image) the image to normalize.
        OUTPUT
            |---- image (np.array) the normalized image.
        """
        arr = np.array(image).astype('float32')
        arr = (arr - arr.min()) / (arr.max() - arr.min())
        arr = (self.vmax - self.vmin) * arr + self.vmin
        return arr

In [31]:
class Cuttings_Dataset(data.Dataset):
    def __init__(self, sample_df, data_path, data_augmentation=True):
        """
        Constructor of the dataset.
        """
        data.Dataset.__init__(self)
        self.sample_df = sample_df
        self.data_path = data_path
        self.data_augmentation = data_augmentation
        
        self.transform =  tf.Compose([tf.Grayscale(num_output_channels=3),
                                    MinMaxNormalization(),
                                    tf.ToTensor(),
                                    tf.Normalize((0.25990713,0.25990713,0.25990713),(0.35051325,0.35051325,0.35051325))])
        
        if data_augmentation:
            self.transform = tf.Compose([tf.Grayscale(num_output_channels=3),
                                    tf.RandomVerticalFlip(p=0.5),
                                    tf.RandomHorizontalFlip(p=0.5),
                                    tf.RandomRotation([-90,90],resample=False, expand=False, center=None, fill=None),
                                    MinMaxNormalization(),
                                    tf.ToTensor(),
                                    tf.Normalize((0.25990713,0.25990713,0.25990713),(0.35051325,0.35051325,0.35051325))])
        
    def __len__(self):
        """
        Get the number of samples in the dataset.
        """
        return self.sample_df.shape[0]


    def __getitem__(self, idx):
        """
        Get an item from the dataset.
        """
        # load image
        im = Image.open('C:/Users/nilso/Documents/EPFL/MA4/PDS Turberg/Models/Convnet/'+self.data_path + self.sample_df.loc[idx,'path'])
        # load label
        label = torch.tensor(self.sample_df.loc[idx,'rock_type'])
        
        im = self.transform(im)
        return im,label

#### Loading data

In [32]:
classes = np.array(['ML', 'MS','BL','GN','OL'])

date = '04_05_20'

df = pd.read_csv('C:/Users/nilso/Documents/EPFL/MA4/PDS Turberg/Models/Convnet/train/csv_'+date+'/train'+date+'_final.csv',index_col=0)

train,val = train_test_split(df,test_size=0.2,random_state=0,stratify=df['rock_type'])
cuttings_datasets = {}

cuttings_datasets['train'] = Cuttings_Dataset(train.reset_index(drop=True),'',data_augmentation=True)
cuttings_datasets['val'] = Cuttings_Dataset(val.reset_index(drop=True),'',data_augmentation=False)
cuttings_datasets['test'] = Cuttings_Dataset(pd.read_csv('C:/Users/nilso/Documents/EPFL/MA4/PDS Turberg/Models/Convnet/test/csv_'+date+'/'+'test'+date+'_final.csv',index_col=0),'',data_augmentation=False)

dataloaders = {x: torch.utils.data.DataLoader(cuttings_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(cuttings_datasets[x]) for x in ['train','val','test']}

In [33]:
# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params

In [34]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [35]:
def initialize_model(num_classes, feature_extract=True, use_pretrained=True):
    model_ft = models.resnet18(pretrained=True)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    return model_ft

In [36]:
model_ft = initialize_model(len(classes))

In [37]:
for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

	 fc.weight
	 fc.bias


In [38]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    loss_train = []
    acc_train = []
    loss_val = []
    acc_val = []
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss_train = 0.0
        running_corrects_train = 0
            
        model.train()# Set model to training mode
        
        # Iterate over data Training
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            inputs.require_grad = True
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            loss = criterion(outputs, labels)

            # backward + optimize
            loss.backward()
            optimizer.step()

            # statistics
            running_loss_train += loss.item() * inputs.size(0)
            running_corrects_train += torch.sum(preds == labels.data)
            
        epoch_loss_train = running_loss_train / dataset_sizes["train"]
        epoch_acc_train = running_corrects_train.double() / dataset_sizes["train"]
        loss_train.append(epoch_loss_train)
        acc_train.append(epoch_acc_train)
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                "Train", epoch_loss_train, epoch_acc_train))
        
        running_loss_val = 0.0
        running_corrects_val = 0
            
        model.eval() # Set model to evaluate mode
        
        # Iterate over data Evaluation
        for inputs, labels in dataloaders["val"]:
            inputs = inputs.to(device)
            labels = labels.to(device)
                
            with torch.no_grad():
                # forward
                outputs = model(inputs)
                preds = torch.argmax(outputs, 1)
                loss = criterion(outputs, labels)   

            # statistics
            running_loss_val += loss.item() * inputs.size(0)
            running_corrects_val += torch.sum(preds == labels.data)

        epoch_loss_val = running_loss_val / dataset_sizes["val"]
        epoch_acc_val = running_corrects_val.double() / dataset_sizes["val"]
        loss_val.append(epoch_loss_val)
        acc_val.append(epoch_acc_val)
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                "Val", epoch_loss_val, epoch_acc_val))
            
        # deep copy the model
        if epoch_acc_val > best_acc:
            best_acc = epoch_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, loss_train, acc_train, loss_val, acc_val

In [39]:
# prediciton
def prediciton(model):
    preds_vec = []
    true_vec = []
    
    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            preds_vec+=preds.tolist()
            true_vec+=labels.tolist()
    return preds_vec, true_vec

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [41]:
device

device(type='cuda', index=0)

In [42]:
date = '29_06_20'

In [43]:
# Get Cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Train Model
list_loss_train = []
list_acc_train = []
list_loss_val = []
list_acc_val = []
list_preds_vec = []
list_true_vec = []

for i in range(5):
    model_ft = initialize_model(len(classes))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model_ft.parameters())

    model_ft = model_ft.to(device)

    model_ft, loss_train, acc_train, loss_val, acc_val = train_model(model_ft, criterion, optimizer,
                       num_epochs=200)
    
    preds_vec, true_vec = prediciton(model_ft)
    
    list_loss_train.append(loss_train)
    list_acc_train.append(acc_train)
    list_loss_val.append(loss_val)
    list_acc_val.append(acc_val)
    list_preds_vec.append(preds_vec)
    list_true_vec.append(true_vec)
    
    torch.save(model_ft, f'results/Resnet_128_padded_{i}_{date}')
# Save Pickles
results = open(f'results/Resnet_results_128_padded_{date}', 'wb')
pickle.dump([[list_loss_train],[list_acc_train],[list_loss_val],[list_acc_val],[list_preds_vec],[list_true_vec]],results)
results.close()

Epoch 0/199
----------
Train Loss: 0.9628 Acc: 0.6298
Val Loss: 0.8618 Acc: 0.6660

Epoch 1/199
----------
Train Loss: 0.7282 Acc: 0.7325
Val Loss: 0.7483 Acc: 0.7120

Epoch 2/199
----------
Train Loss: 0.7073 Acc: 0.7412
Val Loss: 0.7189 Acc: 0.7310

Epoch 3/199
----------
Train Loss: 0.6686 Acc: 0.7432
Val Loss: 0.7357 Acc: 0.7330

Epoch 4/199
----------
Train Loss: 0.6615 Acc: 0.7512
Val Loss: 0.6942 Acc: 0.7380

Epoch 5/199
----------
Train Loss: 0.6283 Acc: 0.7780
Val Loss: 0.7859 Acc: 0.7120

Epoch 6/199
----------
Train Loss: 0.6481 Acc: 0.7568
Val Loss: 0.7899 Acc: 0.6910

Epoch 7/199
----------
Train Loss: 0.6173 Acc: 0.7682
Val Loss: 0.7382 Acc: 0.7160

Epoch 8/199
----------
Train Loss: 0.6288 Acc: 0.7668
Val Loss: 0.7209 Acc: 0.7230

Epoch 9/199
----------
Train Loss: 0.6221 Acc: 0.7625
Val Loss: 0.7373 Acc: 0.7160

Epoch 10/199
----------
Train Loss: 0.6431 Acc: 0.7660
Val Loss: 0.7056 Acc: 0.7300

Epoch 11/199
----------
Train Loss: 0.6127 Acc: 0.7708
Val Loss: 0.6846 Acc

Val Loss: 0.7789 Acc: 0.7220

Epoch 97/199
----------
Train Loss: 0.6065 Acc: 0.7780
Val Loss: 0.7119 Acc: 0.7370

Epoch 98/199
----------
Train Loss: 0.5755 Acc: 0.7885
Val Loss: 0.6768 Acc: 0.7320

Epoch 99/199
----------
Train Loss: 0.5938 Acc: 0.7888
Val Loss: 0.6880 Acc: 0.7320

Epoch 100/199
----------
Train Loss: 0.6090 Acc: 0.7775
Val Loss: 0.6830 Acc: 0.7430

Epoch 101/199
----------
Train Loss: 0.5738 Acc: 0.7868
Val Loss: 0.6788 Acc: 0.7450

Epoch 102/199
----------
Train Loss: 0.6111 Acc: 0.7672
Val Loss: 0.7232 Acc: 0.7410

Epoch 103/199
----------
Train Loss: 0.5900 Acc: 0.7845
Val Loss: 0.6587 Acc: 0.7460

Epoch 104/199
----------
Train Loss: 0.5939 Acc: 0.7815
Val Loss: 0.7015 Acc: 0.7530

Epoch 105/199
----------
Train Loss: 0.5966 Acc: 0.7808
Val Loss: 0.6751 Acc: 0.7360

Epoch 106/199
----------
Train Loss: 0.6087 Acc: 0.7770
Val Loss: 0.6552 Acc: 0.7510

Epoch 107/199
----------
Train Loss: 0.6113 Acc: 0.7805
Val Loss: 0.7061 Acc: 0.7490

Epoch 108/199
----------
Tr

Train Loss: 0.5664 Acc: 0.7963
Val Loss: 0.6747 Acc: 0.7460

Epoch 193/199
----------
Train Loss: 0.5855 Acc: 0.7878
Val Loss: 0.7306 Acc: 0.7380

Epoch 194/199
----------
Train Loss: 0.6001 Acc: 0.7772
Val Loss: 0.6713 Acc: 0.7530

Epoch 195/199
----------
Train Loss: 0.6100 Acc: 0.7810
Val Loss: 0.7781 Acc: 0.7220

Epoch 196/199
----------
Train Loss: 0.5847 Acc: 0.7893
Val Loss: 0.6495 Acc: 0.7580

Epoch 197/199
----------
Train Loss: 0.6037 Acc: 0.7782
Val Loss: 0.7345 Acc: 0.7190

Epoch 198/199
----------
Train Loss: 0.5922 Acc: 0.7840
Val Loss: 0.6641 Acc: 0.7530

Epoch 199/199
----------
Train Loss: 0.5829 Acc: 0.7840
Val Loss: 0.6742 Acc: 0.7460

Training complete in 75m 49s
Best val Acc: 0.775000
Epoch 0/199
----------
Train Loss: 0.9647 Acc: 0.6308
Val Loss: 0.8456 Acc: 0.6800

Epoch 1/199
----------
Train Loss: 0.7431 Acc: 0.7253
Val Loss: 0.7541 Acc: 0.7120

Epoch 2/199
----------
Train Loss: 0.7081 Acc: 0.7333
Val Loss: 0.7692 Acc: 0.7140

Epoch 3/199
----------
Train Loss

Train Loss: 0.5991 Acc: 0.7845
Val Loss: 0.6692 Acc: 0.7410

Epoch 89/199
----------
Train Loss: 0.5911 Acc: 0.7820
Val Loss: 0.6975 Acc: 0.7280

Epoch 90/199
----------
Train Loss: 0.5996 Acc: 0.7815
Val Loss: 0.6394 Acc: 0.7560

Epoch 91/199
----------
Train Loss: 0.5994 Acc: 0.7685
Val Loss: 0.7044 Acc: 0.7410

Epoch 92/199
----------
Train Loss: 0.6116 Acc: 0.7785
Val Loss: 0.6458 Acc: 0.7530

Epoch 93/199
----------
Train Loss: 0.5965 Acc: 0.7825
Val Loss: 0.6733 Acc: 0.7400

Epoch 94/199
----------
Train Loss: 0.6091 Acc: 0.7788
Val Loss: 0.6712 Acc: 0.7460

Epoch 95/199
----------
Train Loss: 0.6061 Acc: 0.7810
Val Loss: 0.6911 Acc: 0.7390

Epoch 96/199
----------
Train Loss: 0.5886 Acc: 0.7810
Val Loss: 0.6813 Acc: 0.7510

Epoch 97/199
----------
Train Loss: 0.6051 Acc: 0.7755
Val Loss: 0.6329 Acc: 0.7420

Epoch 98/199
----------
Train Loss: 0.6081 Acc: 0.7705
Val Loss: 0.6710 Acc: 0.7360

Epoch 99/199
----------
Train Loss: 0.6091 Acc: 0.7728
Val Loss: 0.6680 Acc: 0.7570

Epoc

Train Loss: 0.6003 Acc: 0.7823
Val Loss: 0.6960 Acc: 0.7450

Epoch 185/199
----------
Train Loss: 0.5736 Acc: 0.7843
Val Loss: 0.6796 Acc: 0.7330

Epoch 186/199
----------
Train Loss: 0.5895 Acc: 0.7802
Val Loss: 0.6796 Acc: 0.7500

Epoch 187/199
----------
Train Loss: 0.5884 Acc: 0.7870
Val Loss: 0.6943 Acc: 0.7430

Epoch 188/199
----------
Train Loss: 0.6123 Acc: 0.7758
Val Loss: 0.6874 Acc: 0.7470

Epoch 189/199
----------
Train Loss: 0.5855 Acc: 0.7848
Val Loss: 0.6910 Acc: 0.7390

Epoch 190/199
----------
Train Loss: 0.5927 Acc: 0.7810
Val Loss: 0.6888 Acc: 0.7470

Epoch 191/199
----------
Train Loss: 0.5989 Acc: 0.7845
Val Loss: 0.6801 Acc: 0.7440

Epoch 192/199
----------
Train Loss: 0.5944 Acc: 0.7820
Val Loss: 0.6842 Acc: 0.7480

Epoch 193/199
----------
Train Loss: 0.5925 Acc: 0.7870
Val Loss: 0.6627 Acc: 0.7530

Epoch 194/199
----------
Train Loss: 0.5851 Acc: 0.7843
Val Loss: 0.7065 Acc: 0.7390

Epoch 195/199
----------
Train Loss: 0.6074 Acc: 0.7690
Val Loss: 0.6714 Acc: 0

...

Train Loss: 0.5951 Acc: 0.7792
Val Loss: 0.6461 Acc: 0.7650

Epoch 177/199
----------
Train Loss: 0.6110 Acc: 0.7758
Val Loss: 0.7389 Acc: 0.7280

Epoch 178/199
----------
Train Loss: 0.6178 Acc: 0.7712
Val Loss: 0.6972 Acc: 0.7250

Epoch 179/199
----------
Train Loss: 0.5877 Acc: 0.7885
Val Loss: 0.6626 Acc: 0.7510

Epoch 180/199
----------
Train Loss: 0.5824 Acc: 0.7755
Val Loss: 0.7379 Acc: 0.7320

Epoch 181/199
----------
Train Loss: 0.6119 Acc: 0.7765
Val Loss: 0.6583 Acc: 0.7440

Epoch 182/199
----------
Train Loss: 0.5946 Acc: 0.7833
Val Loss: 0.7099 Acc: 0.7400

Epoch 183/199
----------
Train Loss: 0.5887 Acc: 0.7782
Val Loss: 0.6702 Acc: 0.7600

Epoch 184/199
----------
Train Loss: 0.6161 Acc: 0.7818
Val Loss: 0.6556 Acc: 0.7510

Epoch 185/199
----------
Train Loss: 0.5923 Acc: 0.7863
Val Loss: 0.6806 Acc: 0.7420

Epoch 186/199
----------
Train Loss: 0.5747 Acc: 0.7840
Val Loss: 0.6724 Acc: 0.7400

Epoch 187/199
----------
Train Loss: 0.5759 Acc: 0.7888
Val Loss: 0.6478 Acc: 0

Train Loss: 0.5773 Acc: 0.7855
Val Loss: 0.7012 Acc: 0.7250

Epoch 73/199
----------
Train Loss: 0.6113 Acc: 0.7790
Val Loss: 0.6584 Acc: 0.7490

Epoch 74/199
----------
Train Loss: 0.6009 Acc: 0.7712
Val Loss: 0.7370 Acc: 0.7370

Epoch 75/199
----------
Train Loss: 0.6013 Acc: 0.7795
Val Loss: 0.6587 Acc: 0.7620

Epoch 76/199
----------
Train Loss: 0.6021 Acc: 0.7818
Val Loss: 0.6764 Acc: 0.7390

Epoch 77/199
----------
Train Loss: 0.5765 Acc: 0.7778
Val Loss: 0.6777 Acc: 0.7520

Epoch 78/199
----------
Train Loss: 0.5964 Acc: 0.7840
Val Loss: 0.6743 Acc: 0.7520

Epoch 79/199
----------
Train Loss: 0.5971 Acc: 0.7770
Val Loss: 0.7424 Acc: 0.7390

Epoch 80/199
----------
Train Loss: 0.6073 Acc: 0.7692
Val Loss: 0.7248 Acc: 0.7350

Epoch 81/199
----------
Train Loss: 0.6060 Acc: 0.7762
Val Loss: 0.6755 Acc: 0.7230

Epoch 82/199
----------
Train Loss: 0.5949 Acc: 0.7900
Val Loss: 0.7209 Acc: 0.7330

Epoch 83/199
----------
Train Loss: 0.5586 Acc: 0.7965
Val Loss: 0.6355 Acc: 0.7640

Epoc

Train Loss: 0.5840 Acc: 0.7865
Val Loss: 0.6940 Acc: 0.7550

Epoch 169/199
----------
Train Loss: 0.6205 Acc: 0.7730
Val Loss: 0.6402 Acc: 0.7450

Epoch 170/199
----------
Train Loss: 0.6041 Acc: 0.7800
Val Loss: 0.6845 Acc: 0.7460

Epoch 171/199
----------
Train Loss: 0.5744 Acc: 0.7948
Val Loss: 0.7128 Acc: 0.7210

Epoch 172/199
----------
Train Loss: 0.6059 Acc: 0.7810
Val Loss: 0.6475 Acc: 0.7490

Epoch 173/199
----------
Train Loss: 0.5820 Acc: 0.7840
Val Loss: 0.6919 Acc: 0.7550

Epoch 174/199
----------
Train Loss: 0.6137 Acc: 0.7730
Val Loss: 0.6631 Acc: 0.7560

Epoch 175/199
----------
Train Loss: 0.6080 Acc: 0.7762
Val Loss: 0.6806 Acc: 0.7520

Epoch 176/199
----------
Train Loss: 0.5912 Acc: 0.7848
Val Loss: 0.6862 Acc: 0.7420

Epoch 177/199
----------
Train Loss: 0.5577 Acc: 0.7958
Val Loss: 0.6472 Acc: 0.7350

Epoch 178/199
----------
Train Loss: 0.6074 Acc: 0.7785
Val Loss: 0.6679 Acc: 0.7520

Epoch 179/199
----------
Train Loss: 0.6134 Acc: 0.7802
Val Loss: 0.6778 Acc: 0

Train Loss: 0.5993 Acc: 0.7752
Val Loss: 0.6792 Acc: 0.7570

Epoch 65/199
----------
Train Loss: 0.5954 Acc: 0.7808
Val Loss: 0.6759 Acc: 0.7600

Epoch 66/199
----------
Train Loss: 0.6192 Acc: 0.7720
Val Loss: 0.6692 Acc: 0.7470

Epoch 67/199
----------
Train Loss: 0.5816 Acc: 0.7828
Val Loss: 0.6496 Acc: 0.7570

Epoch 68/199
----------
Train Loss: 0.5591 Acc: 0.7925
Val Loss: 0.6662 Acc: 0.7490

Epoch 69/199
----------
Train Loss: 0.6053 Acc: 0.7692
Val Loss: 0.6483 Acc: 0.7580

Epoch 70/199
----------
Train Loss: 0.6118 Acc: 0.7735
Val Loss: 0.6749 Acc: 0.7610

Epoch 71/199
----------
Train Loss: 0.5883 Acc: 0.7830
Val Loss: 0.7033 Acc: 0.7290

Epoch 72/199
----------
Train Loss: 0.6054 Acc: 0.7820
Val Loss: 0.6996 Acc: 0.7210

Epoch 73/199
----------
Train Loss: 0.5926 Acc: 0.7828
Val Loss: 0.6419 Acc: 0.7500

Epoch 74/199
----------
Train Loss: 0.5829 Acc: 0.7840
Val Loss: 0.6396 Acc: 0.7630

Epoch 75/199
----------
Train Loss: 0.5834 Acc: 0.7868
Val Loss: 0.6467 Acc: 0.7510

Epoc

Train Loss: 0.5997 Acc: 0.7785
Val Loss: 0.6796 Acc: 0.7380

Epoch 161/199
----------
Train Loss: 0.6259 Acc: 0.7682
Val Loss: 0.6698 Acc: 0.7570

Epoch 162/199
----------
Train Loss: 0.5810 Acc: 0.7878
Val Loss: 0.7517 Acc: 0.7330

Epoch 163/199
----------
Train Loss: 0.6202 Acc: 0.7755
Val Loss: 0.6414 Acc: 0.7580

Epoch 164/199
----------
Train Loss: 0.5975 Acc: 0.7812
Val Loss: 0.6380 Acc: 0.7570

Epoch 165/199
----------
Train Loss: 0.5882 Acc: 0.7838
Val Loss: 0.7892 Acc: 0.7040

Epoch 166/199
----------
Train Loss: 0.5769 Acc: 0.7880
Val Loss: 0.7783 Acc: 0.7140

Epoch 167/199
----------
Train Loss: 0.5918 Acc: 0.7860
Val Loss: 0.6468 Acc: 0.7580

Epoch 168/199
----------
Train Loss: 0.5903 Acc: 0.7835
Val Loss: 0.7136 Acc: 0.7420

Epoch 169/199
----------
Train Loss: 0.6056 Acc: 0.7768
Val Loss: 0.6597 Acc: 0.7510

Epoch 170/199
----------
Train Loss: 0.5914 Acc: 0.7808
Val Loss: 0.7289 Acc: 0.7330

Epoch 171/199
----------
Train Loss: 0.5999 Acc: 0.7785
Val Loss: 0.6494 Acc: 0